## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [50]:
import pandas as pd
from pymystem3 import Mystem
from collections import Counter

data = pd.read_csv('/datasets/data.csv')
print(data.head(10))
display(data.info())

   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   
4         0  340266.072047         53   среднее             1   
5         0    -926.185831         27    высшее             0   
6         0   -2879.202052         43    высшее             0   
7         0    -152.779569         50   СРЕДНЕЕ             1   
8         2   -6929.865299         35    ВЫСШЕЕ             0   
9         0   -2188.756445         41   среднее             1   

      family_status  family_status_id gender income_type  debt   total_income  \
0   женат / замужем                 0      F   сотрудник     0  253875.639453   
1   женат / замужем                 0      F   сотрудник     0  112080.014102   
2   женат / замужем                 0    

None

### Вывод

Исходя из полученной информации можно сделать вывод, что некотрые поля содержат некоректные данные

### Шаг 2. Предобработка данных

### Обработка пропусков

In [52]:
def field_nun(data):  #функция для нахождения полей с пропущенными значениями
    columns = data.columns
    for i in columns:
        if data.loc[data[i].isnull()]['education_id'].count() > 0:
            print(i)


field_nun(data)
data['days_employed'] = data['days_employed'].abs()
#data.loc[data['days_employed'] >= data['days_employed'].max() / 24, 'days_employed'] = data['days_employed'] / 24
data.loc[data['gender'] == 'XNA', 'gender'] = 'M'
data.loc[data['dob_years'] == 0, 'dob_years'] = data['dob_years'].mean()
data.loc[(data['children'] == -1) | (data['children'] == 20), 'children'] = data['children'].median()

days_employed_median = data['days_employed'].mean()
total_income_median = data['total_income'].median()

data['days_employed'] = data['days_employed'].fillna(days_employed_median)
data['total_income'] = data['total_income'].fillna(total_income_median)
















### Вывод

Были обнаружены пропущенные значения в полях days_employed и total_income, кроме этого в поле days_employed были найдены большие значения, которые не могут соответсвовать реальности, если они в днях и отрицательные велечины. По моим предположениям, пропущенные значения связаны с тем, что люди не работали и, следовательно, не имели дохода, отрицательные значения я связываю с ошибками ввода данных, очень большие значения связываю с тем, что ввод осуществлялся в часах, а не в днях. Отрицательные значения были заменены значениями модулю велечин, большие значения были переведены из часов в дни. Пропуски в поле days_employed были заменены на среднее арифметическое, так как отклонение велечин от этого среднего не велико. В поле total_income пропуски были заменены медианой, из-за большого разброса значений. В  поле gender был обнаружен баг со значением XNA, значение было изменено на значение М.В поле children найдены значения -1 и 20, они заменены на медиану, так как в случае со среднем получается не целое число, к детям это применить нельзя. В поле dob_years найдены люди с возрастом 0, значение заменено на среднее по полю. Я связываю такие данные с некорректным вводом

### Замена типа данных

In [53]:
float_types_list = ['days_employed', 'total_income', 'dob_years', 'children']

for i in float_types_list:
    data[i] = data[i].astype('int')

display(data.dtypes)



children             int64
days_employed        int64
dob_years            int64
education           object
education_id         int64
family_status       object
family_status_id     int64
gender              object
income_type         object
debt                 int64
total_income         int64
purpose             object
dtype: object

### Вывод

In [54]:
display(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


None

В полях days_employed и total_income были обновлены типы данных, теперь они int64. Использовался метод astype(), так как меняли с типа float.

### Обработка дубликатов

In [55]:
data['education'] = data['education'].str.lower()
print(data.duplicated().sum())
data = data.drop_duplicates().reset_index(drop = True)
print(data.duplicated().sum())




71
0


### Вывод

Так как в таблице остутсвуют поля, которые могут идентифицировать каждую запись, было приянто решение искать полные дубликаты. Было обнаружено и удалено 54 дубликата. Кроме этого в поле education были обнаружены строки, которые имеют один и тот же смысл, но написаны по-разному, например, "высшее" и "ВЫСШЕЕ". Все значения таких дубликатов были приведены к  нижнему регистру.

### Лемматизация

In [56]:
data_purpose = data['purpose'].unique()
print(data_purpose)
lemma_list = []
m = Mystem()
for row in data_purpose:
    lemma = m.lemmatize(row)
    for i in lemma:
        lemma_list.append(i)
print(Counter(lemma_list))


['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']
Counter({' ': 59, '\n': 38, '

### Вывод

В ходе лемматизации обнаружены следующие категории для целей: недвижимость, автомобиль, образование, жилье, свадьба.

### Категоризация данных

In [57]:
def purpose_group(purpose):
    lem_purpose = m.lemmatize(purpose)
    if 'недвижимость' in lem_purpose:
        return 'недвижимость'
    elif 'автомобиль' in lem_purpose:
        return 'автомобиль'
    elif 'образование' in lem_purpose:
        return 'образование'
    elif 'свадьба' in lem_purpose:
        return 'свадьба'
    elif 'жилье' in lem_purpose:
        return 'жилье'
    else: 
        return purpose
    
def children_group(children):
    if children == 0:
        return 'нет детей'
    elif 0 < children <=2:
        return 'есть дети'
    else:
        return 'многодетный'
    
def total_income_group(income):
    if income < 70000:
        return 'до 70000'
    elif 70000 <= income < 120000:
        return 'от 70000 до 12000'
    elif 120000 <= income < 200000:
        return 'от 120000 до 200000'
    else:
        return 'от 200000'
    
    
    
data['purpose_group'] = data['purpose'].apply(purpose_group)
data['children_group'] = data['children'].apply(children_group)
data['total_income_group'] = data['total_income'].apply(total_income_group)
print(data['purpose_group'].value_counts())
print(data['children_group'].value_counts())
print(data['total_income_group'].value_counts())


недвижимость    6351
жилье           4460
автомобиль      4306
образование     4013
свадьба         2324
Name: purpose_group, dtype: int64
нет детей      14214
есть дети       6860
многодетный      380
Name: children_group, dtype: int64
от 120000 до 200000    9543
от 70000 до 12000      5371
от 200000              5066
до 70000               1474
Name: total_income_group, dtype: int64


### Вывод

Также были поделены на категории поля children и total_income.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [58]:
children_pivot = data.pivot_table(index=['children_group'], columns ='debt', values = 'days_employed', aggfunc = 'count')
children_pivot.set_axis(['Вернули в срок', 'Вернули в срок / Все'], axis = 'columns', inplace = True)
children_pivot['Вернули в срок / Все'] = round(children_pivot['Вернули в срок']/data['children_group'].value_counts() * 100,2)
print(children_pivot)

                Вернули в срок  Вернули в срок / Все
children_group                                      
есть дети                 6222                 90.70
многодетный                349                 91.84
нет детей                13142                 92.46


### Вывод

Из полученных результатов делаем вывод, что люди без детей возвращают кредиты вовремя чаще чем многодетные и люди с 1-2 детьми.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [59]:
family_status_pivot = data.pivot_table(index = ['family_status'], columns = 'debt', values = 'days_employed', aggfunc ='count')
family_status_pivot.set_axis(['Вернули в срок', 'Вернули в срок / Все'], axis = 'columns', inplace = True)
family_status_pivot['Вернули в срок / Все'] = round(family_status_pivot['Вернули в срок']/data['family_status'].value_counts() * 100,2)
print(family_status_pivot)

                       Вернули в срок  Вернули в срок / Все
family_status                                              
Не женат / не замужем            2536                 90.25
в разводе                        1110                 92.89
вдовец / вдова                    896                 93.43
гражданский брак                 3763                 90.65
женат / замужем                 11408                 92.45


### Вывод

Сделаем вывод, что люди, которые когда-либо узаконивывали свои отношения возвращают кредиты вовремя чаще чем люди без штампов в паспорте. Стоит выделить вдовцов и вдов, они выглядят надежнее остальных.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [60]:
total_income_group_pivot = data.pivot_table(index=['total_income_group'], columns ='debt', values = 'days_employed', aggfunc = 'count')
total_income_group_pivot.set_axis(['Вернули в срок', 'Вернули в срок / Все'], axis = 'columns', inplace = True)
total_income_group_pivot['Вернули в срок / Все'] = round(total_income_group_pivot['Вернули в срок']/data['total_income_group'].value_counts() * 100,2)
print(total_income_group_pivot)

                     Вернули в срок  Вернули в срок / Все
total_income_group                                       
до 70000                       1373                 93.15
от 120000 до 200000            8711                 91.28
от 200000                      4708                 92.93
от 70000 до 12000              4921                 91.62


### Вывод

Люди с низким зароботком относятся ответственнее к кредитам чем люди со средним заработком. Но от них не отстают богатые.

- Как разные цели кредита влияют на его возврат в срок?

In [61]:
purpose_debt_table_pivot = data.pivot_table(index=['purpose_group'], columns ='debt', values = 'days_employed', aggfunc = 'count')
purpose_debt_table_pivot.set_axis(['Вернули в срок', 'Вернули в срок / Все'], axis = 'columns', inplace = True)
purpose_debt_table_pivot['Вернули в срок / Все'] = round(purpose_debt_table_pivot['Вернули в срок']/data['purpose_group'].value_counts() * 100,2)
print(purpose_debt_table_pivot)


               Вернули в срок  Вернули в срок / Все
purpose_group                                      
автомобиль               3903                 90.64
жилье                    4152                 93.09
недвижимость             5877                 92.54
образование              3643                 90.78
свадьба                  2138                 92.00


### Вывод

Люди, покупающие жилье, к кредитам относятся чуть серьезнее остальных. Возврат кредитов для образования и авто возвращаются не так вовремя, как кредит на свадьбу или недвижимость

### Шаг 4. Общий вывод

В ходе проектной работы были устранены следующие проблемы с данными: были заменыне пропущенные значения в полях на медианы и средние арифметические по полю, изменены типы некоторых полей с float на int, по причине целесообразности, например, в поле о количестве дете нет смысла хранить не целые числа. Кроме этого поля со строковыми данными были приведены к одному регистра для дальнейшего поиска дубликатов, позже дубликаты были обнаружены и удалены. Перед получением основных зависимостей данны были категоризованы по полям целей займа, уровню дохода и наличия/отсутсвия детей. По полученным зависимостям можно сказать следующее: 
##1) люди, у которых нет детей возвращают кредиты вопремя чаще многодетных и ещё чаще людей с 1-2 детьми; 
##2) люди, которые когда-либо узаконивали свои отношения относятся к кредитам серьезнее людей в гражданском браке или пока одних;
##3) люди со средним уровнем дохода не так ответсвенны к возврату кредита, как люди с высоким и низким ежемесячным заработком;
##4) вероятность возврата кредита на авто или образование в срок не такая большая как заемщиков на свадьюу, жильё или недвижимость.


### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.